In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@47e24cff

In [2]:
import spark.implicits._
case class Visits(visit_id: Int, customer_id: Int)
case class Transactions(transaction_id: Int, visit_id: Int, amount: Int)

val data = Seq[(Int, Int)]((1, 23), (2, 9), (4, 30), (5, 54), (6, 96), (7, 54), (8, 54))
val visits = data.toDF("visit_id", "customer_id")
    .as[Visits]
visits.show()

val data2 = Seq[(Int, Int, Int)]((2, 5, 310), (3, 5, 300), (9, 5, 200), (12, 1, 910), (13, 2, 970))
val transactions = data2.toDF("transaction_id", "visit_id", "amount")
    .as[Transactions]
transactions.show()

+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|       1|         23|
|       2|          9|
|       4|         30|
|       5|         54|
|       6|         96|
|       7|         54|
|       8|         54|
+--------+-----------+

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|             2|       5|   310|
|             3|       5|   300|
|             9|       5|   200|
|            12|       1|   910|
|            13|       2|   970|
+--------------+--------+------+



import spark.implicits._

defined class Visits
defined class Transactions
data: Seq[(Int, Int)] = List(
  (1, 23),
  (2, 9),
  (4, 30),
  (5, 54),
  (6, 96),
  (7, 54),
  (8, 54)
)
visits: Dataset[Visits] = [visit_id: int, customer_id: int]
data2: Seq[(Int, Int, Int)] = List(
  (2, 5, 310),
  (3, 5, 300),
  (9, 5, 200),
  (12, 1, 910),
  (13, 2, 970)
)
transactions: Dataset[Transactions] = [transaction_id: int, visit_id: int ... 1 more field]

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

| customer_id | count_no_trans |
|-------------|----------------|
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |

In [3]:
val visitWithPurchase = transactions.groupBy(transactions("visit_id"))
    .agg(sum(transactions("transaction_id")).as("transaction_id"))

visits.join(transactions, Seq("visit_id"), "left_outer")
    .filter((transactions("transaction_id").isNull))
    .select(visits("customer_id"))
    .groupBy(visits("customer_id"))
    .agg(count(visits("customer_id")).as("count_no_trans"))
    .show()


+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         30|             1|
|         96|             1|
|         54|             2|
+-----------+--------------+



visitWithPurchase: DataFrame = [visit_id: int, transaction_id: bigint]

In [4]:
spark.stop()